In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from os import listdir
from os.path import isfile, join


from src.SimilarityArtworks import *
from src.SimilarityUsers import *
from src.ArtworksMatrix import *
from src.UsersMatrix import *
from src.UsersClustering import *
from src.AverageUser import *

In [2]:
import inspect

def inner_classes_list(cls):
    return [cls_attribute.__name__ for cls_attribute in cls.__dict__.values()
            if inspect.isclass(cls_attribute)]
            # and issubclass(cls_attribute, SimilarityFunctionInterface)]
    
    
import functools

def on_button_clicked(b, rs_="some_default_string"):
    fun(rs_)

# button.on_clicked(functools.partial(on_button_clicked, rs_="abcdefg"))

In [73]:
class UserInterface():

    def __init__(self, path='data/'):
      self.path = path
      self.files = [f for f in listdir(self.path) if isfile(join(self.path, f))]
    
### 1. CARGAR DATOS ###
    def loadData(self):
      self.users = pd.read_csv(self.path + self.usersDropdown.value, index_col=0)
      self.items = pd.read_csv(self.path + self.itemsDropdown.value)
      self.users['positive'] = self.users['positive'].apply(eval)
      self.users['negative'] = self.users['negative'].apply(eval)
      self.users['mixed'] = self.users['mixed'].apply(eval)
      self.users_sim_functions = [None]
      self.users_sim_functions.extend(inner_classes_list(SimilarityUsers))
      self.items_sim_functions = [None]
      self.items_sim_functions.extend(inner_classes_list(SimilarityArtworks))
    
    def loadData_create_widgets(self):
      self.completeMessages = widgets.VBox([])
      self.loadDataComplete = widgets.Label(value="- Users and Items data loaded -")
      self.loadDataTitle = widgets.HTML(value="<h2>Select data files</h2>")
      self.usersDropdown = widgets.Dropdown(options=self.files,
                                            value='Prado_users_emotions_OnePolarity.csv',
                                            description='Users:',
                                            disabled=False,
                                            )
      self.itemsDropdown = widgets.Dropdown(options=self.files,
                                            value='Prado_artworks_wikidata.csv',
                                            description='Items:',
                                            disabled=False,
                                            )
      self.loadDataButton = widgets.Button(description="Done", button_style='success')     
      self.loadDataButton.on_click(self.on_loadDataButton_clicked)

    def on_loadDataButton_clicked(self, change):
      with self.out:
        self.out.clear_output()
        self.loadData()
        self.completeMessages.children += (self.loadDataComplete, )
        display(self.completeMessages)
        self.addItemsAtribute_create_widgets()
        display(self.addItemsAtributeTitle)
        display(self.addItemsAtributeButtonBox)
    
### 2. SIMILITUD ITEMS ###
    def computeItemsSim(self):
      self.items_funcs = {x : None for x in self.items.columns}
      self.items_weights = {x : None for x in self.items.columns}  
      
      for atr in self.items_atribute_list:
        if atr.children[0].value not in self.items.columns: # Atributo con fichero externo
          self.items_funcs.update({self.path + atr.children[0].value : atr.children[1].value})
          self.items_weights.update({self.path + atr.children[0].value : atr.children[2].value})
        else:
          self.items_funcs.update({atr.children[0].value : atr.children[1].value})
          self.items_weights.update({atr.children[0].value : atr.children[2].value})
      
      AM = ArtworksMatrix(self.items, function_dict=self.items_funcs, weight_dict=self.items_weights)#, colors_path=self.path + self.addItemsAtributeColor.children[0].value)
      self.itemsMatrix = AM.getSimilarityMatrix()
    
    def addItemsAtribute_create_widgets(self):
      self.items_atribute_list = []
      self.addItemsAtributeComplete = widgets.Label(value="- Items similarity computed -")
      self.addItemsAtributeTitle = widgets.HTML(value="<h2>Add atributes to compute ITEMS similarity</h2>")
      
      self.addItemsAtributeButton = widgets.Button(description="Atribute", icon='plus')
      self.addItemsAtributeButton.on_click(self.on_addItemsAtributeButton_clicked) 
      self.addItemsAtributeButtonExtra = widgets.Button(description="File Atribute", icon='plus')
      self.addItemsAtributeButtonExtra.on_click(self.on_addItemsAtributeButtonExtra_clicked)
      self.addItemsAtributeButtonBox = widgets.HBox([self.addItemsAtributeButton, self.addItemsAtributeButtonExtra])
      
      self.loadItemsAtributeButton = widgets.Button(description="Compute similarity", button_style='success')
      self.loadItemsAtributeButton.on_click(self.on_loadItemsAtributeButton_clicked)
    
    def addItemsAtribute(self):
      aux1 = widgets.Dropdown(options=self.items.columns, value=None, description='Atribute:',disabled=False)
      aux2 = widgets.Dropdown(options=self.items_sim_functions, description='Sim Function:', disabled=False)
      # aux3 = widgets.BoundedFloatText(value=0.0, min=0, max=1, step=0.1, description='Weight:', disabled=False)
      aux3 = widgets.FloatSlider(value=0.0, min=0, max=1, step=0.1, description='Weight:', disabled=False, readout=True, readout_format='.1f')
      
      self.itemsLastAtribute = widgets.HBox([aux1, aux2, aux3])
    
    def addItemsAtributeExtra(self):
      aux1 = widgets.Dropdown(options=self.files, value=None, description='File:', disabled=False)
      aux2 = widgets.Dropdown(options=self.items_sim_functions, value=None, description='Sim Function:', disabled=False)
      # aux3 = widgets.BoundedFloatText(value=0.0, min=0, max=1, step=0.1, description='Weight:', disabled=False)
      aux3 = widgets.FloatSlider(value=0.0, min=0, max=1, step=0.1, description='Weight:', disabled=False, readout=True, readout_format='.1f')
     
      self.itemsLastAtribute = widgets.HBox([aux1, aux2, aux3])
    
    def on_addItemsAtributeButton_clicked(self, change):
      with self.out:
        self.out.clear_output()
        self.addItemsAtribute()
        display(self.completeMessages)
        display(self.addItemsAtributeTitle)
        for atr in self.items_atribute_list:
          display(atr)
        display(self.itemsLastAtribute)
        display(self.addItemsAtributeButtonBox)
        display(self.loadItemsAtributeButton)
        self.items_atribute_list.append(self.itemsLastAtribute)
    
    def on_addItemsAtributeButtonExtra_clicked(self, change):
      with self.out:
        self.out.clear_output()
        self.addItemsAtributeExtra()
        display(self.completeMessages)
        display(self.addItemsAtributeTitle)
        for atr in self.items_atribute_list:
          display(atr)
        display(self.itemsLastAtribute)
        display(self.addItemsAtributeButtonBox)
        display(self.loadItemsAtributeButton)
        self.items_atribute_list.append(self.itemsLastAtribute)    
    
    def on_loadItemsAtributeButton_clicked(self, change):
      with self.out:
        self.out.clear_output()
        display(self.completeMessages)
        print('Computing items similarity...')
        self.computeItemsSim()
        self.out.clear_output()
        self.completeMessages.children += (self.addItemsAtributeComplete, )
        display(self.completeMessages)
        self.addUsersAtribute_create_widgets()
        display(self.addUsersAtributeTitle)
        for atr in self.users_atribute_list:
          display(atr)
        display(self.addUsersAtributeButton)
    
### 3. SIMILITUD USUARIOS ###
    def computeUsersSim(self):
      self.users_funcs = {x : None for x in self.users.columns}
      self.users_weights = {x : None for x in self.users.columns}  
      
      self.users_atribute_list.remove(self.loadUsersAtributePolWeight)
      self.users_atribute_list.remove(self.loadUsersAtributeDemogWeight)
      for atr in self.users_atribute_list:
        self.users_funcs.update({atr.children[0].value : atr.children[1].value})
        self.users_weights.update({atr.children[0].value : atr.children[2].value})
      
      self.users_weights.update({'polarity' : self.loadUsersAtributePolWeight.value, 'demographic' : self.loadUsersAtributeDemogWeight.value}) ######
      
      UM = UsersMatrix(self.users, self.itemsMatrix, function_dict=self.users_funcs, weight_dict=self.users_weights)
      self.usersMatrix = UM.getSimilarityMatrix()
    
    def addUsersAtribute_create_widgets(self):
      self.loadUsersAtributeDemogWeight = widgets.FloatSlider(value=1.0, min=0, max=1, step=0.1, description='Demographic Weight:', disabled=False, readout=True, readout_format='.1f')
      self.loadUsersAtributePolWeight = widgets.FloatSlider(value=0.0, min=0, max=1, step=0.1, description='Polarity Weight:', disabled=False, readout=True, readout_format='.1f')
      self.loadUsersAtributeDemogWeight.style.description_width = 'initial'
      self.loadUsersAtributePolWeight.style.description_width = 'initial'
      
      def loadUsersAtributeDemogWeight_change(change):
        self.loadUsersAtributePolWeight.value = 1 - self.loadUsersAtributeDemogWeight.value
      def loadUsersAtributePolWeight_change(change):
        self.loadUsersAtributeDemogWeight.value = 1 - self.loadUsersAtributePolWeight.value
        
      self.loadUsersAtributeDemogWeight.observe(loadUsersAtributeDemogWeight_change, names='value')
      self.loadUsersAtributePolWeight.observe(loadUsersAtributePolWeight_change, names='value')
      
      self.users_atribute_list = [self.loadUsersAtributeDemogWeight, self.loadUsersAtributePolWeight]
      
      self.addUsersAtributeComplete = widgets.Label(value="- Users similarity computed -")
      self.addUsersAtributeTitle = widgets.HTML(value="<h2>Add atributes to compute USERS similarity</h2>")
      self.addUsersAtributeButton = widgets.Button(description="Atribute", icon='plus')
      self.addUsersAtributeButton.on_click(self.on_addUsersAtributeButton_clicked) 
      self.loadUsersAtributeButton = widgets.Button(description="Compute similarity", button_style='success')
      self.loadUsersAtributeButton.on_click(self.on_loadUsersAtributeButton_clicked)
    
    def addUsersAtribute(self):
      aux1 = widgets.Dropdown(options=self.users.columns, value=None, description='Atribute:', disabled=False)
      aux2 = widgets.Dropdown(options=self.users_sim_functions, description='Sim Function::', disabled=False)
      # aux3 = widgets.BoundedFloatText(value=0.0, min=0, max=1, step=0.1, description='Weight:', disabled=False)
      aux3 = widgets.FloatSlider(value=0.0, min=0, max=1, step=0.1, description='Weight:', disabled=False, readout=True, readout_format='.1f')
      
      self.usersLastAtribute = widgets.HBox([aux1, aux2, aux3])
    
    def on_addUsersAtributeButton_clicked(self, change):
      self.out.clear_output()
      self.addUsersAtribute()
      with self.out:
        display(self.completeMessages)
        display(self.addUsersAtributeTitle)
        for atr in self.users_atribute_list:
          display(atr)
        display(self.usersLastAtribute)
        display(self.addUsersAtributeButton)
        display(self.loadUsersAtributeButton)
        self.users_atribute_list.append(self.usersLastAtribute)
    
    def on_loadUsersAtributeButton_clicked(self, change):
      with self.out:
        self.out.clear_output()
        display(self.completeMessages)
        print('Computing users similarity...')
        self.computeUsersSim()
        self.out.clear_output()
        self.completeMessages.children += (self.addUsersAtributeComplete, )
        display(self.completeMessages)
        self.computeClusters_create_widgets()
        display(self.computeClustersTitle)
        display(self.computeClustersButton)
    
### 4. CLUSTERING ###
    def computeClusters(self):
      users_distances = 1 - self.usersMatrix
      labels = UsersClustering(users_distances).kMedoidsFromMatrix()
      self.users_clustered = self.users.copy()
      self.users_clustered['cluster'] = labels
      
    def computeClusters_create_widgets(self):
      self.computeClustersComplete = widgets.Label(value="- Users clustered -")
      self.computeClustersTitle = widgets.HTML(value="<h2>Compute users clusters</h2>")
      self.computeClustersButton = widgets.Button(description="Compute clusters", button_style='success')
      self.computeClustersButton.on_click(self.on_computeClustersButton_clicked)      
      
    def on_computeClustersButton_clicked(self, change):
      with self.out:
        self.out.clear_output()
        display(self.completeMessages)
        print('Computing users clusters...')
        self.computeClusters()
        self.out.clear_output()
        self.completeMessages.children += (self.computeClustersComplete, )
        display(self.completeMessages)
        self.showExplanation_create_widgets()
        display(self.showExplanationTitle)
        display(self.showExplanationButton)
    
### 5. EXPLICACION ###
    def showExplanation(self):
      users_atributes = [key for key, value in self.users_funcs.items() if value is not None]
      items_atributes = [key for key, value in self.items_funcs.items() if value is not None]
      AU = AverageUser(self.users_clustered, self.items, users_atributes, items_atributes)
      self.explicators = AU.computeAverageUser()
      self.explanation = AU.printExplanation
      
    def showExplanation_create_widgets(self):
      self.showExplanationTitle = widgets.HTML(value="<h2>Clusters explanation</h2>")
      self.showExplanationButton = widgets.Button(description="Show explanation", button_style='success')
      self.showExplanationButton.on_click(self.on_showExplanationButton_clicked)      
      
    def on_showExplanationButton_clicked(self, change):
      with self.out:
        self.out.clear_output()
        self.showExplanation()
        display(self.completeMessages)
        self.explanation()        
    
### MOSTAR TODO ###
    def display_widgets(self):
        self.loadData_create_widgets()
        self.out = widgets.Output()  # this is the output widget in which the df is displayed
        
        display(self.out)
        with self.out:
          display(self.loadDataTitle)
          display(widgets.VBox(
                              [
                                  self.usersDropdown,
                                  self.itemsDropdown,
                                  self.loadDataButton,
                              ]
                          )
                 )


## RUN ##
UserInterface().display_widgets()

Output()